<a href="https://colab.research.google.com/github/CarlosMendez1997Col/Advanced-AI-ML-DL-NN-LLM-Generative-Algorithms/blob/main/Web%20Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Development of Advanced AI-ML-DL-NN-LLM-Generative Algorithms

### `Deep, automated search for analyzing calls for proposals and tenders at the Stockholm Environment Institute (SEI)`

## Web and data Mining: Scraping, Crawling and Parsing

#### Code Developed by Carlos Mendez, Research Associate I






# Install/import libraries and packages

In [1]:
!pip install selenium # Dynamic Scraping
#%pip install -q google-colab-selenium
#!apt-get update
#!apt install chromium-chromedriver

!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb
!pip install selenium webdriver-manager

!pip install requests # Static Scraping
!pip install beautifulsoup4 #Parser in HTML

!pip install scrapy # Massive Scraping
!pip install webdriver-manager #Management ChromeDriver
!pip install playwright # Alternative to Selenium
!pip install pydantic # Validate data

!pip install psycopg2 # Connection with PostgreSQL
!pip install elasticsearch # Advanced Queries and Index
!pip install firecrawl # Massive Crawling

!pip install transformers #Semantic Embeddings
!pip install sentence-transformers #Transform data into NLP

!pip install fastapi # Use API REST
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  at-spi2-core gsettings-desktop-schemas libatk-bridge2.0-0 libatk1.0-0
  libatk1.0-data libatspi2.0-0 libvulkan1 libxcomposite1 libxtst6
  mesa-vulkan-drivers session-migration
The following NEW packages will be installed:
  at-spi2-core google-chrome-stable gsettings-desktop-schemas

In [2]:
# --- 2. Configurar Selenium ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_argument("--headless")  # obligatorio en Colab
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

import requests
import sys
import time

from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from fastapi import FastAPI
from firecrawl import FirecrawlApp
from pydantic import BaseModel, Field
from playwright.sync_api import sync_playwright

# Ministerio de Ciencia Tecnología e Innovación

In [10]:
BASE_URL = "https://minciencias.gov.co"

def scrape_minciencias_headers():
    url = f"{BASE_URL}/convocatorias/todas"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0 Safari/537.36"
    }
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    results = []
    for row in soup.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) >= 5:
            numero = cols[0].get_text(strip=True)

            # El título y el enlace de detalle
            link_tag = cols[1].find("a")
            titulo = link_tag.get_text(strip=True) if link_tag else cols[1].get_text(strip=True)
            rel_url = link_tag["href"] if link_tag and link_tag.has_attr("href") else None
            url_detalle = BASE_URL + rel_url if rel_url else None

            descripcion = cols[2].get_text(strip=True)
            recursos = cols[3].get_text(strip=True)
            apertura = cols[4].get_text(strip=True)

            # Capturar cualquier link a PDF dentro de la fila
            pdf_links = []
            for a in row.find_all("a", href=True):
                href = a["href"]
                if href.lower().endswith(".pdf"):
                    if href.startswith("/"):
                        href = BASE_URL + href
                    pdf_links.append(href)

            results.append({
                "numero": numero,
                "titulo": titulo,
                "descripcion": descripcion,
                "recursos": recursos,
                "fecha_apertura": apertura,
                "url_detalle": url_detalle,
                "pdfs": pdf_links
            })
    return results

if __name__ == "__main__":
    data = scrape_minciencias_headers()
    for item in data:
        print(item)


# Ejemplo: MinCiencias
minciencias_data = [
    {
        "institucion": "MinCiencias",
        "titulo_convocatoria": item["titulo"],
        "objetivo_descripcion": item["descripcion"],
        "fecha_inicio": item["fecha_apertura"],
        "fecha_cierre": None,  # si no está disponible
        "recursos": item["recursos"],
        "link": item["url_detalle"]
    }
    for item in scrape_minciencias_headers()
]

{'numero': '51', 'titulo': 'CONVOCATORIA PARA EL FORTALECIMIENTO DE CAPACIDADES DE CIENCIA, TECNOLOGÍA E INNOVACIÓN EN EL DEPARTAMENTO DE CÓRDOBA ALINEADO CON LOS RETOS ESTRATÉGICOS DE CTeI DEL PLAN BIENAL 2025-2026', 'descripcion': 'Fortalecer las capacidades de Ciencia, Tecnología e Innovación (CTeI) en el Departamento de Córdoba, mediante el desarrollo de proyectos de convergencia regional que impulsen la productividad y competitividad de acuerdo a las vocaciones territoriales dirigidas a atender las demandas territoriales de', 'recursos': '$17.154.450.476', 'fecha_apertura': 'Martes, Diciembre 30, 2025', 'url_detalle': 'https://minciencias.gov.co/node/9215', 'pdfs': []}
{'numero': '45', 'titulo': 'FORTALECIMIENTO Y CREACIÓN DE NUEVOS CENTROS E INSTITUTOS DE INVESTIGACIÓN, CENTROS DE DESARROLLO TECNOLÓGICO Y CENTROS DE CIENCIA', 'descripcion': 'Aumentar la capacidad de investigación de los Centros e Institutos de Investigación, Centros de Desarrollo Tecnológico y Centros de Ciencia,

# Ministerio de Ambiente y Desarrollo Sostenible

In [11]:
def scrape_minambiente():
    url = "https://www.minambiente.gov.co/convocatorias"  # Ajustar si cambia
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    data = []
    # Seleccionamos todas las filas de la tabla
    for row in soup.select("table tr"):
        cols = row.find_all("td")
        if not cols:
            continue  # saltar filas sin celdas (ej. encabezados)

        # Ajustar índices según estructura real de la tabla
        titulo = cols[0].get_text(strip=True)
        fecha = cols[1].get_text(strip=True) if len(cols) > 1 else None
        enlace_tag = cols[0].find("a")
        enlace = enlace_tag["href"] if enlace_tag else None

        data.append({
            "fuente": "MinAmbiente",
            "titulo": titulo,
            "fecha": fecha,
            "url": enlace
        })
    return data

# Prueba rápida
if __name__ == "__main__":
    resultados = scrape_minambiente()
    for r in resultados:
        print(r)

# Ejemplo: MinAmbiente
minambiente_data = [
    {
        "institucion": "MinAmbiente",
        "titulo_convocatoria": r["titulo"],
        "objetivo_descripcion": None,
        "fecha_inicio": r["fecha"],
        "fecha_cierre": None,
        "recursos": None,
        "link": r["url"]
    }
    for r in scrape_minambiente()
]


{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 1', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-01-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 2', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-02-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 3', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-03-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 4', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-04-de-2021.pdf'}
{'fuente': 'MinAmbiente', 'titulo': 'Convocatoria Externa N° 5', 'fecha': 'Publicación28/12/2021', 'url': 'https://www.minambiente.gov.co/wp-content/uploads/2021/12/Convocatoria-05-de-2021.pdf'}
{'fuente': 'MinAmbiente',

# Fundación Natura Colombia

In [12]:
url = "https://natura.org.co/convocatorias/"
resp = requests.get(url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

for block in soup.select("div.box-text.text-left"):
    titulo = block.find("h5", class_="post-title")
    resumen = block.find("p", class_="from_the_blog_excerpt")
    fecha = block.find("div", class_="post-date")
    if titulo:
        print("Título:", titulo.get_text(strip=True))
    if resumen:
        print("Resumen:", resumen.get_text(strip=True))
    if fecha:
        print("Fecha:", fecha.get_text(strip=True))
    print("-" * 60)

# Ejemplo: Natura (usa los prints que capturaste)
natura_data = []
for block in soup.select("div.box-text.text-left"):
    titulo = block.find("h5", class_="post-title")
    resumen = block.find("p", class_="from_the_blog_excerpt")
    fecha = block.find("div", class_="post-date")
    natura_data.append({
        "institucion": "Fundación Natura",
        "titulo_convocatoria": titulo.get_text(strip=True) if titulo else None,
        "objetivo_descripcion": resumen.get_text(strip=True) if resumen else None,
        "fecha_inicio": fecha.get_text(strip=True) if fecha else None,
        "fecha_cierre": None,
        "recursos": None,
        "link": "https://fundacionnatura.org.co/convocatorias"
    })

Título: Convocatoria Profesional control y aseguramiento de la calidad SINGEI – 4CNCC
Resumen: En el marco del proyecto “Cuarta Comunicación Nacional de Cambio Climático de Colombia y Primer...
------------------------------------------------------------
Título: Convocatoria Profesional en vulnerabilidad para el proyecto 4CNCC
Resumen: En el marco del proyecto “Cuarta Comunicación Nacional de Cambio Climático de Colombia y Primer...
------------------------------------------------------------


# Fundación para la Conservación y el Desarrollo

In [13]:
url = "https://fcds.org.co/oportunidades-laborales/"
resp = requests.get(url)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

# Cada convocatoria está en un bloque con h3 y p
for block in soup.select("div.elementor-widget-container"):
    titulo = block.find("h3")
    objetivo = block.find("p")
    if titulo and objetivo:
        print("Título:", titulo.get_text(strip=True))
        print("Objetivo:", objetivo.get_text(strip=True))
        print("-" * 60)

# Ejemplo: FCDS
fcds_data = []
for block in soup.select("div.elementor-widget-container"):
    titulo = block.find("h3")
    objetivo = block.find("p")
    if titulo and objetivo:
        fcds_data.append({
            "institucion": "FCDS",
            "titulo_convocatoria": titulo.get_text(strip=True),
            "objetivo_descripcion": objetivo.get_text(strip=True),
            "fecha_inicio": None,
            "fecha_cierre": None,
            "recursos": None,
            "link": "https://fcds.org.co/oportunidades-laborales/"
        })

Título: Profesional en análisis geoespacial de dinámicas de motores de transformación en bioma amazónico y fronteras
Objetivo: Prestar sus servicios profesionales para generar y analizar información geoespacial y documental relacionada a las trasformaciones territoriales, presentes en bioma amazónico nacional y sus fronteras.
------------------------------------------------------------
Título: Profesional en análisis geoespacial de dinámicas de motores de transformación en bioma amazónico y fronteras
Objetivo: Prestar sus servicios profesionales para generar y analizar información geoespacial y documental relacionada a las trasformaciones territoriales, presentes en bioma amazónico nacional y sus fronteras.
------------------------------------------------------------
Título: Consultoría Especializada en SST año 2026
Objetivo: Contratar una persona jurídica que preste servicios profesionales de outsourcing para la actualización documental, implementación y mejora continua del sistema de

# FontAgro

In [14]:
url = "https://fontagro.org/es/iniciativas/convocatorias"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

convocatorias = []

# Buscar títulos
for title in soup.find_all("h4", class_="initiative-title"):
    titulo = title.get_text(strip=True)

    # Buscar el enlace asociado (si existe)
    link_tag = title.find_next("a")
    link = link_tag["href"] if link_tag else None

    # Buscar la imagen asociada
    img_tag = title.find_next("img", class_="initiative-image")
    img = img_tag["src"] if img_tag else None

    convocatorias.append({
        "titulo": titulo,
        "link": f"https://fontagro.org{link}" if link else None,
        "imagen": img
    })

# Mostrar resultados
for c in convocatorias:
    print(c)

# Ejemplo: FontAgro
fontagro_data = []
for c in convocatorias:  # tu lista construida en el scraping
    fontagro_data.append({
        "institucion": "FontAgro",
        "titulo_convocatoria": c["titulo"],
        "objetivo_descripcion": None,
        "fecha_inicio": None,
        "fecha_cierre": None,
        "recursos": None,
        "link": c["link"]
    })


{'titulo': 'Convocatoria 2026', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2026', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2025_f0388b44d4.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2025', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2025', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2024_160edcb44a.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2024', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2024', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2023_9d9acb3d59.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2023', 'link': 'https://fontagro.org/es/iniciativas/convocatorias/convocatoria-2023', 'imagen': '/_next/image?url=https%3A%2F%2Fstrapi-fontagro.nyc3.digitaloceanspaces.com%2Fcon_2022_d6c386cec2.jpg&w=640&q=100'}
{'titulo': 'Convocatoria 2022', 'link': 'https://fon

# NESsT

In [20]:
# --- 3. Scraping NESsT BambooHR ---
driver.get("https://nesst.bamboohr.com/careers")

# Esperar a que aparezcan las vacantes
elements = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.fab-LinkUnstyled"))
)

# Guardar títulos y enlaces (evita stale elements)
vacantes = [{"titulo": el.text, "link": el.get_attribute("href")} for el in elements]

jobs = []
for v in vacantes:
    driver.get(v["link"])
    try:
        descripcion_tag = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#content, div#description, div.jobDescription"))
        )
        descripcion = descripcion_tag.text
    except:
        descripcion = None

    jobs.append({
        "titulo": v["titulo"],
        "link": v["link"],
        "descripcion": descripcion
    })

driver.quit()

# --- 4. Mostrar resultados ---
print("Vacantes encontradas:", len(jobs))
for j in jobs:
    print("="*80)
    print("Título:", j["titulo"])
    print("Link:", j["link"])
    if j["descripcion"]:  # solo si no es None
        print("Descripción:\n", j["descripcion"][:300], "...\n")
    else:
        print("Descripción: No disponible\n")

# Ejemplo: NESsT
nesst_data = []
for j in jobs:  # tu lista construida con Selenium
    nesst_data.append({
        "institucion": "NESsT",
        "titulo_convocatoria": j["titulo"],
        "objetivo_descripcion": j["descripcion"],
        "fecha_inicio": None,
        "fecha_cierre": None,
        "recursos": "Inversión social y donaciones",
        "link": j["link"]
    })

MaxRetryError: HTTPConnectionPool(host='localhost', port=41275): Max retries exceeded with url: /session/00aabcca286006f580086192d8d66b22/url (Caused by NewConnectionError("HTTPConnection(host='localhost', port=41275): Failed to establish a new connection: [Errno 111] Connection refused"))

# Database with the information

In [18]:
# --- 2. Unificar todas las listas ---
convocatorias = minciencias_data + minambiente_data + natura_data + fcds_data + fontagro_data

# --- 3. Crear DataFrame ---
df = pd.DataFrame(convocatorias)

# --- 4. Visualización dinámica con ipywidgets ---
dropdown = widgets.Dropdown(
    options=["Todas"] + df["institucion"].unique().tolist(),
    description="Institución:"
)

def mostrar_convocatorias(change):
    display(df if change.new == "Todas" else df[df["institucion"] == change.new])

dropdown.observe(mostrar_convocatorias, names="value")
display(dropdown)


Dropdown(description='Institución:', options=('Todas', 'MinCiencias', 'MinAmbiente', 'Fundación Natura', 'FCDS…

,institucion,titulo_convocatoria,objetivo_descripcion,fecha_inicio,fecha_cierre,recursos,link
0,MinCiencias,CONVOCATORIA PARA EL FORTALECIMIENTO DE CAPACI...,"Fortalecer las capacidades de Ciencia, Tecnolo...","Martes, Diciembre 30, 2025",None,$17.154.450.476,https://minciencias.gov.co/node/9215
1,MinCiencias,FORTALECIMIENTO Y CREACIÓN DE NUEVOS CENTROS E...,Aumentar la capacidad de investigación de los ...,"Martes, Diciembre 30, 2025",None,$200.000.000.000,https://minciencias.gov.co/node/9216
2,MinCiencias,Convocatoria para la formación de capital huma...,Incrementar la disponibilidad de capital human...,"Jueves, Noviembre 20, 2025",None,$877.800.000,https://minciencias.gov.co/node/9164
3,MinCiencias,CONVOCATORIA EN INVESTIGACIÓN BÁSICA,Generar nuevo conocimiento en las líneas temát...,"Martes, Octubre 14, 2025",None,$237.398.000.000,https://minciencias.gov.co/node/9123
4,MinCiencias,CONVOCATORIA COLOMBIA INTELIGENTE: INFRAESTRUC...,"Impulsar la infraestructura, el desarrollo cie...","Martes, Octubre 14, 2025",None,$630.000.000.000,https://minciencias.gov.co/node/9125


,institucion,titulo_convocatoria,objetivo_descripcion,fecha_inicio,fecha_cierre,recursos,link
5,MinAmbiente,Convocatoria Externa N° 1,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
6,MinAmbiente,Convocatoria Externa N° 2,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
7,MinAmbiente,Convocatoria Externa N° 3,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
8,MinAmbiente,Convocatoria Externa N° 4,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
9,MinAmbiente,Convocatoria Externa N° 5,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
10,MinAmbiente,Convocatoria Externa N° 6,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
11,MinAmbiente,Convocatoria Externa N° 7,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
12,MinAmbiente,Convocatoria Externa N° 8,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
13,MinAmbiente,Convocatoria Externa N° 9,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...
14,MinAmbiente,Convocatoria Externa N° 10,None,Publicación28/12/2021,None,None,https://www.minambiente.gov.co/wp-content/uplo...


,institucion,titulo_convocatoria,objetivo_descripcion,fecha_inicio,fecha_cierre,recursos,link
24,Fundación Natura,Convocatoria Profesional control y aseguramien...,En el marco del proyecto “Cuarta Comunicación ...,None,None,None,https://fundacionnatura.org.co/convocatorias
25,Fundación Natura,Convocatoria Profesional en vulnerabilidad par...,En el marco del proyecto “Cuarta Comunicación ...,None,None,None,https://fundacionnatura.org.co/convocatorias


,institucion,titulo_convocatoria,objetivo_descripcion,fecha_inicio,fecha_cierre,recursos,link
26,FCDS,Profesional en análisis geoespacial de dinámic...,Prestar sus servicios profesionales para gener...,None,None,None,https://fcds.org.co/oportunidades-laborales/
27,FCDS,Profesional en análisis geoespacial de dinámic...,Prestar sus servicios profesionales para gener...,None,None,None,https://fcds.org.co/oportunidades-laborales/
28,FCDS,Consultoría Especializada en SST año 2026,Contratar una persona jurídica que preste serv...,None,None,None,https://fcds.org.co/oportunidades-laborales/


,institucion,titulo_convocatoria,objetivo_descripcion,fecha_inicio,fecha_cierre,recursos,link
29,FontAgro,Convocatoria 2026,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
30,FontAgro,Convocatoria 2025,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
31,FontAgro,Convocatoria 2024,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
32,FontAgro,Convocatoria 2023,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
33,FontAgro,Convocatoria 2022,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
34,FontAgro,Convocatoria 2021,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
35,FontAgro,Convocatoria 2020,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
36,FontAgro,Convocatoria 2019 Productividad,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
37,FontAgro,Convocatoria 2019 AGTECH,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
38,FontAgro,Convocatoria 2018,None,None,None,None,https://fontagro.org/es/iniciativas/convocator...
